In [3]:
# 1. 설치
import sys
!{sys.executable} -m pip install fastparquet

# 2. 엔진 바꿈
import pandas as pd
db = pd.read_parquet(r"/Users/rachel/Downloads/무제 폴더/codes/3. tokenize/df_final_preprocessed.parquet", engine='fastparquet')

In [5]:
db = pd.read_parquet(r"/Users/rachel/Downloads/무제 폴더/codes/3. tokenize/df_final_preprocessed.parquet", engine='fastparquet')

print(db.head())

   doc_id        date                                            content  \
0       0  2025-11-14            14일 서울 중구 하나은행 본점 딜링룸 현황판에 코스피가 표시돼 있다.   
1       0  2025-11-14           나흘 연속 상승했던 코스피가 3 대 급락하며 4000선을 간신히 지켰다.   
2       0  2025-11-14  간밤 미국 셧다운 해제에도 불구하고 금리인하 기대감이 약화한 데다 고평가 논란이 불...   
3       0  2025-11-14  14일 코스피는 전일 대비 159.06포인트 3.81 내린 4011.57에 거래를 ...   
4       0  2025-11-14  이날 지수는 전일 대비 108.72포인트 2.61 급락한 4061.91에 출발한 뒤...   

                                              tokens category source  
0  [중구/NNG;본점/NNG;딜링룸/NNG;현황/NNG;판/NNG, 코스피/NNG;표...       뉴스   매일경제  
1  [연속/NNG;상승/NNG;하/VV;코스피/NNG;급락/NNG, 선/NNG;간신히/...       뉴스   매일경제  
2  [간밤/NNG;폐쇄/NNG;해제/NNG;불구/NNG;금리/NNG, 인하/NNG;기대...       뉴스   매일경제  
3       [코스피/NNG;대비/NNG;포인트/NNG;내리/VV;거래/NNG, 마치/VV]       뉴스   매일경제  
4  [지수/NNG;대비/NNG;포인트/NNG;급락/NNG;출발/NNG, 뒤/NNG;낙폭...       뉴스   매일경제  


In [ ]:
from collections import Counter
import numpy as np


# 1. 파일 불러오기
load_path = r"/Users/rachel/Downloads/무제 폴더/codes/3. tokenize/df_final_preprocessed.parquet"
# 저장할 새파일 경로
save_path = r"/Users/rachel/Downloads/무제 폴더/codes/3. tokenize/prepro delete under 15.parquet"

print("1단계 : 원본 파일 불러오기")

db = pd.read_parquet(load_path, engine='fastparquet')



# 2. 전체 단어 갯수 세기
print("2단계 : 단어 등장 횟수 측정")

all_tokens = []

for row in db['tokens']:
    if isinstance(row, (list, np.ndarray)):
        all_tokens.extend(row)

# 카운트
token_counts = Counter(all_tokens)

print(f"    >>총 발견된 단어 종류 : {len(token_counts):,} 개")


# 3. 15회 이상 등장 단어 목록 생성
print("3단계 : 15번 이상 등장 목록 만들기")

valid_tokens = {word for word, count in token_counts.items() if count >= 15}

deleted_count = len(token_counts) - len(valid_tokens)
print(f"    >>15회 이상 등장 단어 : {len(valid_tokens):,} 개")
print(f"    >>15회 미만 등장으로 삭제될 단어 : {deleted_count:,} 개")


# 4. 필터링
print("4단계: 15회 미만 등장 단어 삭제")

def clean_tokens(token_list):
    if not isinstance(token_list, (list, np.ndarray)):
        return []
    
    return [t for t in token_list if t in valid_tokens]

# 덮어쓰기
db['tokens'] = db['tokens'].apply(clean_tokens)


# 5. 저장 및 검증
print("5단계: 파일 저장 중")
db.to_parquet(save_path, engine='fastparquet')
print(f"    >>저장 완료: {save_path}")

print("\n최종 검증")
# 저장된 파일을 다시 읽어서 확인
df_check = pd.read_parquet(save_path, engine='fastparquet')

check_list = []
for row in df_check['tokens']:
    if isinstance(row, (list, np.ndarray)):
        check_list.extend(row)

final_counts = Counter(check_list)

if not final_counts:
    print("     >>결과: 데이터 비었음")
else:
    min_count = min(final_counts.values())
    min_word = min(final_counts, key=final_counts.get)

    print(f"    >>가장 적게 등장한 횟수: {min_count} 번")
    print(f"    >>단어 예: '{min_word}'")

    if min_count >= 15:
        print("     >>결과: 성공. 15번 미만인 단어 전부 제거 완료.")
    else:
        print(f"    >>결과: 실패. 아직 {min_count}번 등장한 단어가 남아있음.")

1단계 : 원본 파일 불러오기
2단계 : 단어 등장 횟수 측정
    >>총 발견된 단어 종류 : 9,153,263 개
3단계 : 15번 이상 등장 목록 만들기
    >>15회 이상 등장 단어 : 27,693 개
    >>15회 미만 등장으로 삭제될 단어 : 9,125,570 개
4단계: 15회 미만 등장 단어 삭제
5단계: 파일 저장 중
    >>저장 완료: /Users/rachel/Downloads/무제 폴더/codes/3. tokenize/prepro delete under 15.parquet

최종 검증
    >>가장 적게 등장한 횟수: 15 번
    >>단어 예: '서비스/NNG;지원/NNG;하/VV'
     >>결과: 성공. 15번 미만인 단어 전부 제거 완료.
